# Système Expert Médical Complet

Ce notebook présente la conception, l’implémentation et l’explication détaillée d’un système expert médical moderne, intégrant :
- La collecte multi-sources de données (SQLite, JSON, API)
- Un pipeline ETL robuste (nettoyage, fusion, validation)
- Des analyses et visualisations avancées
- La fouille de motifs fréquents (Apriori, FP-Growth, Eclat)
- Un moteur d’inférence basé sur les règles d’association
- Un système d’apprentissage incrémental
- Une API Flask pour le diagnostic et l’apprentissage
- Un déploiement facilité par Docker

## 🗺️ Schéma général du pipeline

▶️ **Exécuter cette cellule pour afficher le pipeline**

In [3]:
from graphviz import Digraph
dot = Digraph(comment='Pipeline Système Expert Médical', format='png')
dot.attr(rankdir='LR')
dot.node('A', 'Sources de données\n(SQLite, JSON, API)')
dot.node('B', 'Nettoyage & Fusion')
dot.node('C', 'Données structurées\n(CSV fusionné)')
dot.node('D', 'Fouille de motifs\n(Apriori, FP-Growth, Eclat)')
dot.node('E', 'Moteur d\'inférence\nRègles d\'association')
dot.node('F', 'API Flask\n(Diagnostic, Apprentissage)')
dot.edges(['AB', 'BC', 'CD', 'DE', 'EF'])
dot.render('pipeline_expert_medical', view=True)
dot

ModuleNotFoundError: No module named 'graphviz'

## 1️⃣ Introduction

La médecine moderne s’appuie de plus en plus sur la donnée pour améliorer le diagnostic, la prise en charge et la personnalisation des traitements. Ce projet vise à construire un système expert médical, capable de :
- Diagnostiquer à partir de symptômes
- Recommander des traitements/médicaments
- Apprendre de nouveaux cas en continu
- Être interfaçable via une API web

**Justification des choix** :  
- Multi-sources = richesse et robustesse des connaissances  
- Fouille de motifs = découverte de patterns médicaux pertinents  
- API = intégration facile et déploiement cloud

## 2️⃣ Collecte de données multi-sources

**Sources utilisées** :
- `data/raw/clean_medical_records.csv` : données structurées fusionnées
- `data/raw/patients.json` : cas patients non structurés
- `data/raw/openfda_meds.csv` : base médicaments (issue d’OpenFDA ou équivalent)
- (Optionnel) Base SQLite pour historique ou logs

▶️ **RUN** : Charger toutes les sources de données

In [ ]:
import pandas as pd
import json
import sqlite3

# Chargement du CSV fusionné (structuré)
df_structured = pd.read_csv('data/raw/clean_medical_records.csv')
print("Exemple données structurées :")
display(df_structured.head())

# Chargement du JSON (patients non structurés)
with open('data/raw/patients.json', 'r', encoding='utf-8') as f:
    patients_json = json.load(f)
print("Exemple patient JSON :")
print(patients_json[0])

# Chargement de la base médicaments
df_meds = pd.read_csv('data/raw/openfda_meds.csv')
print("Exemple médicaments :")
display(df_meds.head())

# (Optionnel) Connexion à une base SQLite
# conn = sqlite3.connect('data/raw/medical_data.db')
# df_sql = pd.read_sql_query("SELECT * FROM patients", conn)
# print("Exemple données SQLite :")
# display(df_sql.head())
# conn.close()

## 3️⃣ Nettoyage, fusion et validation des données

**Explications** :
- Gestion des valeurs manquantes
- Normalisation des symptômes (ex : minuscules, accents, synonymes…)
- Fusion des sources (CSV, JSON, SQLite)
- Validation (unicité, cohérence, types)

▶️ **RUN** : Nettoyer et fusionner les données

In [ ]:
# Nettoyage des symptômes (exemple)
def normalize_symptoms(symptom_list):
    import unidecode
    return [unidecode.unidecode(s.lower().strip()) for s in symptom_list]

# Application sur le CSV
df_structured['symptoms_norm'] = df_structured['symptoms'].apply(eval).apply(normalize_symptoms)

# Ajout des patients JSON à la structure
json_patients_df = pd.DataFrame(patients_json)
json_patients_df['symptoms_norm'] = json_patients_df['symptoms'].apply(normalize_symptoms)

# Fusion (exemple)
df_all = pd.concat([df_structured, json_patients_df], ignore_index=True)
print(f"Nombre total de cas fusionnés : {len(df_all)}")
display(df_all.head())

## 4️⃣ Visualisations et analyses descriptives

**Distribution des diagnostics :**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10,4))
sns.countplot(y='diagnosis', data=df_all, order=df_all['diagnosis'].value_counts().index)
plt.title('Distribution des diagnostics')
plt.xlabel('Nombre de cas')
plt.ylabel('Diagnostic')
plt.show()

**Cooccurrence des symptômes (matrice de chaleur) :**

▶️ **RUN** : Visualiser la cooccurrence des symptômes

In [ ]:
from itertools import combinations
from collections import Counter
import numpy as np

# Création de la matrice de cooccurrence
all_symptoms = [s for sublist in df_all['symptoms_norm'] for s in sublist]
unique_symptoms = list(set(all_symptoms))
symptom_pairs = Counter()

for symptoms in df_all['symptoms_norm']:
    for pair in combinations(sorted(set(symptoms)), 2):
        symptom_pairs[pair] += 1

matrix = np.zeros((len(unique_symptoms), len(unique_symptoms)))
for (s1, s2), count in symptom_pairs.items():
    i, j = unique_symptoms.index(s1), unique_symptoms.index(s2)
    matrix[i, j] = count
    matrix[j, i] = count

plt.figure(figsize=(12,10))
sns.heatmap(matrix, xticklabels=unique_symptoms, yticklabels=unique_symptoms, cmap='YlGnBu')
plt.title('Matrice de cooccurrence des symptômes')
plt.show()

## 5️⃣ Historique et comparaison des algorithmes de motifs fréquents

*(À compléter dans les prochaines cellules : explications, implémentations, comparaisons)*